In [1]:
import numpy as np

In [10]:
hex2bin_map = {
   "0":"0000",
   "1":"0001",
   "2":"0010",
   "3":"0011",
   "4":"0100",
   "5":"0101",
   "6":"0110",    
   "7":"0111",
   "8":"1000",
   "9":"1001",
   "a":"1010",
   "b":"1011",
   "c":"1100",
   "d":"1101",
   "e":"1110",
   "f":"1111",
}

In [102]:
def import_txt(file_str):
    results = []
    with open(file_str) as inputfile:
        for line in inputfile:
##        print(line.strip()[-4:])
            results.append(line.strip()[-4:].split(','))
    track_results = results[14:len(results)-16]
    n_track = int(np.floor(len(track_results)/28))
    track_v = np.array(track_results).reshape(n_track,-1)
    return track_v, results

In [5]:
def overlap_m(track_v, verb):
    num_track = len(track_v)
    re_matrix = list(np.array(range(num_track)))
    for i in range(num_track):
        for j in range(num_track):
            if i > j:
                check = (track_v[i,12:] == track_v[j,12:])
                hits = np.floor(sum(check[0:2])/2)+np.floor(sum(check[2:4])/2)+np.floor(sum(check[4:6])/2)+np.floor(sum(check[6:8])/2)+sum(check[8:16])
                if hits>6:
                    if verb=="v":
                        print(hits,i,j, check)
                    re_matrix[j] = np.append(re_matrix[j], [i])
    return re_matrix

In [6]:
def cleanup_index(re_matrix):
    num_track = len(re_matrix)
    index_v = np.ones(num_track)
    for i in range(num_track):
        if type(re_matrix[i])==np.ndarray:
            len_in = len(re_matrix[i])
        else:
            len_in = 1
        for j in range(1,len_in):
            index_v[re_matrix[i][j]] = 0
    return index_v
##        re_matrix[re_matrix[i][j]] = [-1]

In [13]:
def hex2bin(hex_num):
    bin_num = ""
    for i in range(1,4):
        bin_num = bin_num+hex2bin_map[hex_num[i]]
    return bin_num

In [26]:
def layer_map_sum(layer_map):
    lmap_v = np.zeros(12)
    lmap_sum = 0
    for i in range(0,12):
        lmap_v[i] = int(layer_map[i])
##    print(lmap_v)
    lmap_sum = sum(lmap_v)
    return lmap_sum

In [74]:
def track_picking(rc_v, track_v, verb):
    if isinstance(rc_v, np.ndarray):
        sum_v = np.zeros(len(rc_v))
        chisq_v = []
        for i in range(len(rc_v)):
            sum_v[i] = layer_map_sum(hex2bin(track_v[rc_v[i],3]))
            chisq_v.append(track_v[rc_v[i],6])
            if verb=="v":
                print(hex2bin(track_v[rc_v[i],3]), rc_v[i])
        l_max = np.where(sum_v == max(sum_v))[0]
        if verb=="v":
            print(chisq_v, l_max)
        select_index = l_max[0]
        if len(l_max)>1:
            chisq_min = '7c00'
            #chisq_min = chisq_v[0]
            for j in range(len(l_max)):
                if verb=="v":
                    print(chisq_min)
                if chisq_min > chisq_v[l_max[j]]:
                    chisq_min = chisq_v[l_max[j]]
            select_index = chisq_v.index(chisq_min)
        track_index = rc_v[select_index]
    else: 
        track_index = rc_v
    ##print(track_index)
    return track_index

In [123]:
def HitWarrior(track_v, verbal="s"):
    matrix = overlap_m(track_v, verbal)
    index_v = cleanup_index(matrix)
    re_cluster = np.delete(matrix,np.where(index_v < 0.05), axis=0)
    HW_output_index = np.ones((len(re_cluster),), dtype=int)*200
    for i in range(len(re_cluster)):
        HW_output_index[i] = track_picking(re_cluster[i], track_v, verbal)
    return re_cluster, HW_output_index

In [124]:
track_v, _ = import_txt('ftksim_3tracks.txt')

In [125]:
rc, HW_out = HitWarrior(track_v)

In [126]:
list(HW_out)

[16, 1, 18, 5, 8, 25]

In [127]:
list(rc)

[array([ 0,  6,  7,  9, 12, 16]),
 1,
 array([ 2,  3,  4, 17, 18, 20, 21, 22]),
 5,
 array([ 8, 10, 11, 13, 14]),
 array([15, 19, 23, 24, 25, 26, 27])]

In [111]:
hw_v, _ = import_txt('ftksim_3tracks_hwout.txt')

In [134]:
for i in range(len(HW_out)):
    for j in range(len(hw_v)):
        if sum(track_v[HW_out[i], 12:28] == hw_v[j, 12:28]) > 10:
            print("in_track #",HW_out[i]," --> out_track #",j, sep="")
            print(track_v[HW_out[i]] == hw_v[j])

in_track #16 --> out_track #1
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
in_track #1 --> out_track #3
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
in_track #18 --> out_track #2
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
in_track #5 --> out_track #4
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
in_track #8 --> out_track #5
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True 

In [112]:
hw_v[0]

array(['9bda', '064e', '0000', '0f7f', '0000', '0000', '49f7', '3584',
       'd6b6', '13fe', '3e75', '8de7', '0307', '0780', '0760', '120c',
       '0760', '13cc', '0760', '15f4', '048e', '04b6', '0016', '07f1',
       '01c6', '01ec', '0392', '036a'], dtype='<U4')

In [85]:
hw_v[0] == track_v[25]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [43]:
hw_v[5] == track_v[8]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [58]:
hw_v[1] == track_v[16]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [79]:
hw_v[2] == track_v[18]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [83]:
hw_v[3] == track_v[1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [84]:
hw_v[4] == track_v[5]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [80]:
list(hw_v[2])

['9bda',
 '1c4e',
 '0000',
 '0fff',
 '0000',
 '0000',
 '4e46',
 '3766',
 'd6c4',
 '2944',
 '3f0f',
 '0ccd',
 '4313',
 '2498',
 '07a0',
 '166c',
 '07c0',
 '00c8',
 '0800',
 '1484',
 '0594',
 '05b0',
 '0030',
 '001a',
 '00ae',
 '10c1',
 '00e0',
 '10cf']

In [88]:
for i in range(28):
    if i == 12:
        print("----------------------------------------------")
    print(hw_v[:,i])

['9bda' '9bda' '9bda' '9bda' '9bda' '9bda']
['064e' '0019' '1c4e' '174c' '104e' '0097']
['0000' '0000' '0000' '0000' '0000' '0000']
['0f7f' '0f7f' '0fff' '0ffb' '0fcf' '0f3f']
['0000' '0000' '0000' '0000' '0000' '0000']
['0000' '0000' '0000' '0000' '0000' '0000']
['49f7' '4b23' '4e46' '420c' '4d6b' '45e7']
['3584' '35da' '3766' '3935' '391a' '369c']
['d6b6' 'd6b9' 'd6c4' 'c870' 'd0e6' 'd6b6']
['13fe' 'ad5a' '2944' 'b7f7' 'b90a' '1131']
['3e75' '3e48' '3f0f' '3fef' '3fa0' '3e79']
['8de7' '8b84' '0ccd' '0e98' '0fe9' '083d']
----------------------------------------------
['0307' '0299' '4313' '02fd' '13c6' '0307']
['0780' '186c' '2498' '0970' '115c' '0790']
['0760' '06b0' '07a0' '1938' '039c' '0760']
['120c' '0088' '166c' '03f0' '0148' '01c0']
['0760' '0630' '07c0' '0640' '0910' '0760']
['13cc' '10e4' '00c8' '03e2' '18a4' '02a0']
['0760' '05d0' '0800' '03c0' '05fc' '0760']
['15f4' '0198' '1484' '01c0' '0540' '1374']
['048e' '02b2' '0594' '05ba' '0300' '0226']
['04b6' '0276' '05b0' '0596' 

In [ ]:
for i in range(28):
    print(hw_v[2,i], track_v[2,i], track_v[3,i], track_v[4,i], track_v[17,i], track_v[18,i])

In [ ]:
print(hex2bin(track_v[0,3]),hex2bin(track_v[6,3]),hex2bin(track_v[7,3]),hex2bin(track_v[9,3]),hex2bin(track_v[12,3]),hex2bin(track_v[16,3]))

In [ ]:
print(hex2bin(track_v[8,3]),hex2bin(track_v[10,3]),hex2bin(track_v[11,3]),hex2bin(track_v[13,3]),hex2bin(track_v[14,3]))

In [ ]:
track_v[16]

In [50]:
hw_v[1]

array(['9bda', '0019', '0000', '0f7f', '0000', '0000', '4b23', '35da',
       'd6b9', 'ad5a', '3e48', '8b84', '0299', '186c', '06b0', '0088',
       '0630', '10e4', '05d0', '0198', '02b2', '0276', '0358', '638c',
       '0432', '0404', '251c', '3545'], dtype='<U4')

In [52]:
track_v[8]

array(['9bda', '0097', '0000', '0f3f', '0000', '0000', '45e7', '369c',
       'd6b6', '1131', '3e79', '083d', '0307', '0790', '0760', '01c0',
       '0760', '02a0', '0760', '1374', '0226', '024c', '0213', '01ef',
       '020a', '022e', '01d4', '01b0'], dtype='<U4')

In [ ]:
track_v[16]==hw_v[1]

In [ ]:
hex_x = np.array([0x420c])

In [ ]:
hex_x.astype(float)

In [ ]:
np.argmin([track_v[8,6],track_v[10,6],track_v[11,6],track_v[13,6],track_v[14,6]])

In [ ]:
np.where(test_v == min(test_v))

In [ ]:
test_list = [track_v[15,6],track_v[15,6],track_v[19,6],track_v[23,6],track_v[24,6],track_v[25,6],track_v[26,6]]

In [ ]:
test_v = np.array(test_list)

In [ ]:
min(test_v)

In [ ]:
test_v

In [ ]:
np.where(test_v == min(test_v))[0]

In [ ]:
[track_v[8,5],track_v[10,5],track_v[11,5],track_v[13,5],track_v[14,5]]

In [ ]:
[track_v[8,4],track_v[10,4],track_v[11,4],track_v[13,4],track_v[14,4]]

In [ ]:
track_v[8]

In [ ]:
hw_v[5]==track_v[8]